In [1]:
from requests import get
from bs4 import BeautifulSoup

import csv
import pandas as pd

from time import sleep
from random import randint

In [2]:
# url = 'https://tabs.ultimate-guitar.com/tab/bill-withers/aint-no-sunshine-chords-468744'

In [3]:
def parse_title(url):
    artist_and_title = url.split('tab/')[1]  

    if '/' in artist_and_title:
        artist = artist_and_title.split('/')[0]  
        title  = artist_and_title.split('/')[1]
        title  = title.split('-ch')[0]
    else:
        artist = 'unknown'
        title = 'unknown'
    
    return artist, title

In [4]:
def scrape(url, skill): 
    output_list = []
    chordlist   = []
    
    sleep(randint(1,4))
    response = get(url)
    html_soup = BeautifulSoup(response.text, 'lxml')

    lessSoupy = html_soup.find('div', class_='js-store')
    stringySoup = str(lessSoupy)

    splitSoup = stringySoup.split('[ch]')[1:-1]

    for ielement in range(0, len(splitSoup)):
        chordlist.append(str(splitSoup[ielement]).split('[/ch')[0])
        
    artist_name, song_name = parse_title(url)

    output_list.append(("Song", song_name))
    output_list.append(("Artist", artist_name))

#     allChords = ("All Chords", chords)
    #output_list.append(allChords)

    chords = set(chordlist)
    chords = ",".join(chords)
    chords = ("Chords", chords)
    output_list.append(chords)
#     output_list.append(allChords)

    skill_out = ("Label", skill)
    output_list.append(skill_out)

    song_info = dict(output_list)

    return song_info

In [8]:
# Time to get the chords
from os import listdir
from os.path import isfile, join
import string

nPages     = 10
skill      = ['novice', 'intermediate', 'advanced']
linksPath  = '/Users/aaronlevi/Documents/Insight2020/chordRec/ug_song_links/'
linksFiles = [f for f in listdir(linksPath) if isfile(join(linksPath, f))]
# linksFiles.remove('.DS_Store')
# alpha = string.ascii_lowercase

# load the txt file of links for each skill level and each page and SCRAPE
for iLevel in range(0,len(skill)):
    for iPage in range(1,nPages+1):
        # get list of links
        # link_df = pd.read_csv('ug_song_links/intermediate_page1_links.txt', sep=' ', header=None, names=['song_links'])
        link_df = pd.read_csv('ug_song_links/'+skill[iLevel]+'_'+str(iPage)+'_links.txt', sep=' ', header=None, names=['song_links'])

        # scrape links one by one for chords, save out a dictionary when successful
        for iSong in range(0, len(link_df)):
            url = link_df['song_links'][iSong]
            song_info = scrape(url, skill[iLevel])

            if song_info['Chords']:
                # set up name string
                tmpName = [song_info['Artist'] + ' - ' + song_info['Song']]
                fname = ' ' 
                fname = fname.join(tmpName)

                # open, write, save a csv under "Aritst - Song"
#                 with open('ug_chords/'+fname+'.csv', 'w') as f:
                with open('ug_chords10/'+fname+'.csv', 'w') as f:
                    for key in song_info.keys():
                        f.write("%s,%s\n"%(key,song_info[key]))